In [ ]:

%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal as sig

os.chdir(os.path.dirname(os.getcwd()))

import spatial_metrics.helper_functions as hf
import spatial_metrics.detect_peaks as dp

import spatial_metrics.cell_model_base as cs_model
import spatial_metrics.spatial_metrics_1d_calcium_base as pl


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_speeds_with_accumulating_acceleration(timevector,sampling_rate):
    # Initialize accelerations array starting at 0
    accelerations = np.zeros(timevector.shape[0])
    
    # Calculate speeds based on accumulated accelerations
    speeds = np.zeros(num_steps)
    for i in range(1, num_steps):
        # Add a new Gaussian value to the previous acceleration
        new_acceleration = np.random.normal(0, 1)  # Gaussian value with mean 0 and std dev 0.5
        accelerations[i] = accelerations[i - 1] + new_acceleration
        
        # Calculate the new speed based on accumulated acceleration
        new_speed = speeds[i - 1] + accelerations[i - 1]
        
        # Ensure speeds remain positive
        if new_speed < 0:
            accelerations[i] *= -1  # Change sign of acceleration
            speeds[i] = 0  # Set speed to 0
        else:
            speeds[i] = new_speed
    
    # Clip speeds to ensure they remain within the range of 0 to 100 cm/s
    speeds = np.clip(speeds, 0, 100)
    
    return accelerations,speeds

# Simulate speeds over a period of 10 seconds with 1000 steps
duration = 10  # seconds
num_steps = 1000

accelerations,speeds = simulate_speeds_with_accumulating_acceleration(duration, num_steps)

# Plotting the histogram of simulated speeds
plt.figure(figsize=(8, 5))
plt.hist(speeds, bins=50, density=True)
plt.xlabel('Speed (cm/s)')
plt.ylabel('Probability Density')
plt.title('Histogram of Simulated Speeds')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_sine_acceleration(time_vector, sampling_rate, initial_frequency, max_variation, noise_amplitude):   
    frequency_changes = np.random.uniform(-max_variation, max_variation, len(time_vector))
    frequency = initial_frequency + np.cumsum(frequency_changes)
    frequency = np.clip(frequency, 1, 5)  # Limit frequency between 1 and 10 Hz
    frequency_with_noise = frequency + noise_amplitude * np.random.randn(len(time_vector))
    phase = 2 * np.pi * np.cumsum(frequency_with_noise) / sampling_rate

    amplitude = amplitude_variation * np.random.normal(0,20,len(time_vector))
    signal = amplitude*np.sin(phase)
    
    return signal

# Parameters
duration = 100  # seconds
sampling_rate = 10  # Hz
dt = 1/sampling_rate
time_vector = np.linspace(0, duration, duration*sampling_rate)

amplitude_variation = 1*dt
initial_frequency = 0  # Initial frequency of the sine wave (Hz)
max_variation = 0.1*dt  # Maximum frequency variation per time step (Hz)
noise_amplitude = 2 # Amplitude of noise affecting the frequency


# Generate signal with random walk evolving frequency, noise, and frequency limits
signal = generate_sine_acceleration(time_vector, sampling_rate, initial_frequency, max_variation, noise_amplitude)

# Plotting the signal with randomly evolving frequency, noise, and frequency limits
plt.figure(figsize=(8, 5))
plt.plot(time_vector, signal)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Sine Wave with Randomly Evolving Frequency, Noise, and Frequency Limits')
plt.grid(True)
plt.show()


In [ ]:
accelerations = signal
# Calculate speeds based on accumulated accelerations
speeds = np.zeros(time_vector.shape[0])
for i in range(1, time_vector.shape[0]):
  
    # Calculate the new speed based on accumulated acceleration
    new_speed = speeds[i - 1] + accelerations[i - 1]
    
    # Ensure speeds remain positive
    if new_speed < 0:
        speeds[i] = 0  # Set speed to 0
    elif new_speed > 100:
        speeds[i] = 100
        
    else:
        speeds[i] = new_speed

plt.figure(figsize=(8, 5))
plt.plot(time_vector,accelerations,'k')
plt.xlabel('Speed (cm/s)')
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(time_vector,speeds,'k')
plt.xlabel('Speed (cm/s)')
plt.show()



In [ ]:


def generate_random_walk(sampling_rate=100., duration=500, head_direction_srate=10.,
                         speed_srate=5., rho1=1., sigma=0.02, mu_e=0., smooth_points=0.1, **kwargs):

    if smooth_points == 0:
        smooth_points = 1

    environment_edges = kwargs.get('environment_edges', [[0, 100], [0, 100]])

    sampling_rate = float(sampling_rate)
    duration = float(duration)
    total_points = int(duration * sampling_rate)

    total_points_head = int(duration * head_direction_srate)
    head_direction = np.zeros(total_points_head)
    
    head_direction_sigma = math.pi / 4
    head_direction_mu = 0
    random_phases = np.random.normal(head_direction_mu, head_direction_sigma, total_points_head)

    for t in range(1, total_points_head):
        head_direction[t] = np.angle(np.exp(1j * (head_direction[t - 1] + random_phases[t - 1])))

    x_original = np.linspace(0, duration, head_direction.shape[0])
    interpol_func = interpolate.interp1d(x_original, head_direction, kind='cubic')
    x_new = np.linspace(0, duration, total_points)
    head_direction_new = interpol_func(x_new)

    total_points_spd = int(duration * speed_srate)
    speeds = np.random.exponential(100. / sampling_rate, total_points_spd)
    speeds_new = np.interp(x_new, np.linspace(0, duration, speeds.shape[0]), speeds)

    y_coordinates = np.zeros(total_points)
    x_coordinates = np.zeros(total_points)

    x_coordinates[0] = random.uniform(*environment_edges[0])
    y_coordinates[0] = random.uniform(*environment_edges[1])

    epsy = np.random.normal(mu_e, sigma, total_points)
    epsx = np.random.normal(mu_e, sigma, total_points)

    for t in range(1, total_points):
        y_coordinates[t] = y_coordinates[t - 1] + speeds_new[t] * np.sin(head_direction_new[t]) + rho1 * epsy[t]
        x_coordinates[t] = x_coordinates[t - 1] + speeds_new[t] * np.cos(head_direction_new[t]) + rho1 * epsx[t]

        # Ensure the animal stays within the environment
        if x_coordinates[t] >= environment_edges[0][1] or x_coordinates[t] <= environment_edges[0][0] \
            or y_coordinates[t] >= environment_edges[1][1] or y_coordinates[t] <= environment_edges[1][0]:
            
            # head_direction_new[t:] += math.pi
            head_direction_new[t:] = np.angle(np.exp(1j*(head_direction_new[t:] + math.pi)))
            
            y_coordinates[t] = y_coordinates[t-1] + speeds_new[t] * np.sin(head_direction_new[t])
            x_coordinates[t] = x_coordinates[t-1] + speeds_new[t] * np.cos(head_direction_new[t])
    
    
    x_coordinates = hf.gaussian_smooth_1d(x_coordinates.squeeze(), smooth_points)
    y_coordinates = hf.gaussian_smooth_1d(y_coordinates.squeeze(), smooth_points)
    
    np.clip(x_coordinates, *environment_edges[0], out=x_coordinates)
    np.clip(y_coordinates, *environment_edges[1], out=y_coordinates)

    time_vector = np.linspace(0, total_Time, total_points)
    speed, speed_smoothed = hf.get_speed(x_coordinates, y_coordinates, time_vector, sigma_points=smooth_points)

    return x_coordinates, y_coordinates, speed, speed_smoothed, time_vector
